In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [34]:
d180 = np.pi
d360 = d180 * 2
d90 = d180 / 2


P = np.array([[0, d90, 50, 0],
              [120, 0, 0, 0],
              [170, 0, 0, 0]], dtype=np.float64)

P_ranges = [(-d360, d360),
            (0, d180),
            (-d90, 0)]

In [35]:
def test_angles(Ranges, thetas, tolerance=1e-4):
    for (r_min, r_max), theta in zip(Ranges, thetas):
        if not (r_min - tolerance <= theta <= r_max + tolerance):
            return False
    return True

In [ ]:
def forward(Params, Ranges, thetas):
    
    # if not test_angles(Ranges, thetas):
    #    raise ValueError("ERROR: Incorrect value ranges")

    params = Params.copy()
    params[:, 3] = thetas
    
    T = np.eye(4)
    nodes = []
    for i in range(params.shape[0]):
        a, alpha, d, theta = params[i, :]
        
        ct, st = np.cos(theta), np.sin(theta)
        ca, sa = np.cos(alpha), np.sin(alpha)

        Ti = np.array([[ct, -st * ca, st * sa, a * ct],
                       [st, ct * ca, -ct * sa, a * st],
                       [0, sa, ca, d],
                       [0, 0, 0, 1]], dtype=np.float64)
        T = T @ Ti
    
    return T

In [37]:
def verify_solution(Params, Ranges, thetas, target, tolerance=1e-4):
    # if not test_angles(Ranges, thetas):
    #    print(Ranges, thetas)
    #    return False

    forpath = forward(Params, Ranges, thetas)
    pos = forpath[:3, 3]
    
    error = np.linalg.norm(pos - target)

    return error < tolerance


def cos_theorem(target, a1, a2):
    cos_val = (a1 ** 2 + a2 ** 2 - target ** 2) / (2 * a1 * a2)
    cos_val = np.clip(cos_val, -1.0, 1.0)
    return np.arccos(cos_val)


def solution(Params, Ranges, x, y, z):
    a1, a2 = Params[1:, 0]
    d = Params[0, 2]
    z_eff = z - d

    r_xy = np.sqrt(x ** 2 + y ** 2)
    c = np.sqrt(r_xy ** 2 + z_eff ** 2)
    
    tolerance = 1e-6
    min_reach = abs(a1 - a2)
    max_reach = a1 + a2
    min_z = d - max_reach
    max_z = d + max_reach
    
    print(min_reach, max_reach, min_z, max_z, sep="\n")
    
    if not (min_reach - tolerance <= c <= max_reach + tolerance):
        raise ValueError("ERROR: Point is unreachable")
    
    thetas1 = [
        np.arctan2(y, x),
        np.arctan2(-y, -x)
    ]

    thetas3 = [
        np.pi - cos_theorem(c, a1, a2),
        np.pi + cos_theorem(c, a1, a2)
    ]
    
    alpha = np.arctan2(z_eff, r_xy)
    beta = cos_theorem(a2, a1, c)
    
    thetas2 = [
        alpha - beta,
        alpha + beta
    ]
    
    res = []
    for theta1 in thetas1:
        for theta2 in thetas2:
            for theta3 in thetas3:
                if verify_solution(Params, Ranges, np.array([theta1, theta2, theta3]), np.array([x, y, z])):
                    res.append([theta1, theta2, theta3])
    
    return res
    

In [40]:
# after_rotation = forward(P, [0.0, d90, 0.0])
X = 0
Y = 280
Z = 80
print(np.sqrt(X ** 2 + Y ** 2 + Z ** 2))
sol = solution(P, P_ranges, X, Y, Z)
print(sol)
after_rotation = forward(P, P_ranges, sol[0])
print(after_rotation)

x, y, z = after_rotation[:-1, -1]
print(f"Position: x={x:.6f} y={y:.6f} z={z:.6f}")

291.20439557122074
50.0
290.0
-240.0
340.0
[[np.float64(1.5707963267948966), np.float64(-0.18131977440149022), np.float64(0.48995732625372845)], [np.float64(1.5707963267948966), np.float64(0.3947911196997615), np.float64(5.793227980925858)]]
[[ 3.97390210e-17 -7.69389961e-17  1.00000000e+00  1.53080850e-14]
 [ 9.52748312e-01 -3.03760849e-01 -6.12323400e-17  2.80000000e+02]
 [ 3.03760849e-01  9.52748312e-01  6.12323400e-17  8.00000000e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Position: x=0.000000 y=280.000000 z=80.000000


In [39]:
def visualize_position(T):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))